# DataSens E1_v1 — 04_crud_tests

- Objectifs: démontrer CRUD sur `document` + QA simples
- Prérequis: 03_ingest_sources
- Vérifs: counts, update, delete, doublons
- Guide: docs/GUIDE_TECHNIQUE_E1.md



In [ ]:
# ============================================================
# 🎬 DASHBOARD NARRATIF - OÙ SOMMES-NOUS ?
# ============================================================
# Ce dashboard vous guide à travers le pipeline DataSens E1
# Il montre la progression et l'état actuel des données
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import matplotlib.patches as mpatches

print("\n" + "="*80)
print("🎬 FIL D'ARIANE VISUEL - PIPELINE DATASENS E1")
print("="*80)

# Créer figure dashboard
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)
ax.set_xlim(0, 10)
ax.set_ylim(0, 6)
ax.axis('off')

# Étapes du pipeline
etapes = [
    {"nom": "📥 COLLECTE", "status": "✅", "desc": "Sources brutes"},
    {"nom": "☁️ DATALAKE", "status": "✅", "desc": "MinIO Raw"},
    {"nom": "🧹 NETTOYAGE", "status": "🔄", "desc": "Déduplication"},
    {"nom": "💾 ETL", "status": "⏳", "desc": "PostgreSQL"},
    {"nom": "📊 ANNOTATION", "status": "⏳", "desc": "Enrichissement"},
    {"nom": "📦 EXPORT", "status": "⏳", "desc": "Dataset IA"}
]

# Couleurs selon statut
colors = {
    "✅": "#4ECDC4",
    "🔄": "#FECA57", 
    "⏳": "#E8E8E8"
}

# Dessiner timeline
y_pos = 4
x_start = 1
x_spacing = 1.4

for i, etape in enumerate(etapes):
    x_pos = x_start + i * x_spacing
    
    # Cercle étape
    circle = plt.Circle((x_pos, y_pos), 0.25, color=colors[etape["status"]], zorder=3)
    ax.add_patch(circle)
    ax.text(x_pos, y_pos, etape["status"], ha='center', va='center', fontsize=14, fontweight='bold', zorder=4)
    
    # Nom étape
    ax.text(x_pos, y_pos - 0.6, etape["nom"], ha='center', va='top', fontsize=11, fontweight='bold')
    ax.text(x_pos, y_pos - 0.85, etape["desc"], ha='center', va='top', fontsize=9, style='italic')
    
    # Flèche vers prochaine étape
    if i < len(etapes) - 1:
        ax.arrow(x_pos + 0.3, y_pos, x_spacing - 0.6, 0, 
                head_width=0.1, head_length=0.15, fc='gray', ec='gray', zorder=2)

# Titre narratif
ax.text(5, 5.5, "🎯 PROGRESSION DU PIPELINE E1", ha='center', va='center', 
        fontsize=16, fontweight='bold', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Légende
legend_elements = [
    mpatches.Patch(facecolor='#4ECDC4', label='Terminé'),
    mpatches.Patch(facecolor='#FECA57', label='En cours'),
    mpatches.Patch(facecolor='#E8E8E8', label='À venir')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

# Statistiques rapides (si disponibles)
stats_text = "\n📊 SNAPSHOT ACTUEL :\n"
try:
    # Essayer de charger des stats si base disponible
    stats_text += "   • Pipeline en cours d'exécution...\n"
except:
    stats_text += "   • Démarrage du pipeline...\n"

ax.text(5, 1.5, stats_text, ha='center', va='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.title("🎬 FIL D'ARIANE VISUEL - Accompagnement narratif du jury", 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 Le fil d'Ariane vous guide étape par étape à travers le pipeline")
print("   Chaque visualisation s'inscrit dans cette progression narrative\n")



> Notes:
> - Ouverture de la base SQLite et définition de 4 helpers: CREATE/READ/UPDATE/DELETE.
> - `list_documents()` s’appuie sur pandas pour un affichage rapide.
> - Ordre de test: lister → insérer → relister → mettre à jour → relister → supprimer.
> - But: valider le cycle CRUD avant les analytics.


In [ ]:
# DataSens E1_v1 - 04_crud_tests
# 🛠️ Utilitaires CRUD pour `document` + test
import sqlite3
from pathlib import Path

import pandas as pd

# 1) Ouvrir la base SQLite
# Détection robuste du dossier projet (remonte jusqu'à trouver le dossier avec notebooks/ et docs/)
current = Path.cwd()
PROJECT_ROOT = None
# Remonter jusqu'à trouver un dossier qui contient à la fois notebooks/ et docs/
while current != current.parent:
    if (current / "notebooks").exists() and (current / "docs").exists():
        PROJECT_ROOT = current
        break
    current = current.parent
else:
    # Fallback: utilise le répertoire courant si pas trouvé
    PROJECT_ROOT = Path.cwd()

DB_PATH = PROJECT_ROOT / "datasens" / "datasens.db"
print(f"📂 Racine projet détectée : {PROJECT_ROOT}")

# Vérifier que le schéma existe
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='document'")
if not cur.fetchone():
    conn.close()
    print("⚠️ La table 'document' n'existe pas.")
    print("   Ordre d'exécution requis:")
    print("   1. 02_schema_create.ipynb (créer les tables)")
    print("   2. 03_ingest_sources.ipynb (insérer des données de test)")
    print("   3. 04_crud_tests.ipynb (tester CRUD)")
    raise Exception("Schéma manquant: exécutez 02_schema_create.ipynb et 03_ingest_sources.ipynb avant 04_crud_tests.ipynb")
print(f"✅ Base SQLite prête : {DB_PATH}")

# 2) Petit helper pour lire des SELECT dans un DataFrame (affichage pratique)
def sql_df(query, params=None):
    return pd.read_sql_query(query, conn, params=params)

# 3) CREATE: insérer un document rattaché à un flux (traçabilité)
def add_document(id_flux, titre, texte, langue, date_publication):
    cur.execute(
        "INSERT INTO document (id_flux, titre, texte, langue, date_publication) VALUES (?, ?, ?, ?, ?)",
        (id_flux, titre, texte, langue, date_publication),
    )
    conn.commit()

# 4) READ: lister et lire un document

def list_documents():
    return sql_df("SELECT id_doc, titre, langue, date_publication FROM document;")

def get_document(id_doc):
    return sql_df("SELECT * FROM document WHERE id_doc = ?", params=[id_doc])

# 5) UPDATE: ne modifie que les champs fournis (mise à jour partielle)

def update_document(id_doc, titre=None, texte=None, langue=None, date_publication=None):
    updates, params = [], []
    if titre is not None:
        updates.append("titre = ?"); params.append(titre)
    if texte is not None:
        updates.append("texte = ?"); params.append(texte)
    if langue is not None:
        updates.append("langue = ?"); params.append(langue)
    if date_publication is not None:
        updates.append("date_publication = ?"); params.append(date_publication)
    if updates:
        params.append(id_doc)
        sql = f"UPDATE document SET {', '.join(updates)} WHERE id_doc = ?"
        cur.execute(sql, params)
        conn.commit()

# 6) DELETE: suppression d’un document
def delete_document(id_doc):
    cur.execute("DELETE FROM document WHERE id_doc = ?", (id_doc,))
    conn.commit()


📂 Racine projet détectée : c:\Users\Utilisateur\Desktop\DataSens\notebooks\datasens_E1_v1
✅ Base SQLite prête : c:\Users\Utilisateur\Desktop\DataSens\notebooks\datasens_E1_v1\datasens\datasens.db


In [ ]:
# 🧪 Test CRUD sur `document` avec visualisations
import matplotlib.pyplot as plt

print("📄 Documents initiaux :")
df_before = list_documents()
display(df_before)

# Visualisation avant CRUD
count_before = len(df_before)
plt.figure(figsize=(12, 5))

# Graphique 1 : Évolution du nombre de documents
stages = ["Avant CRUD", "Après CREATE", "Après UPDATE", "Après DELETE"]
counts = [count_before]

print("\n➕ Ajout d'un document…")
add_document(1, "Inflation 2024", "Hausse des prix ressentie", "fr", "2024-09-15")
df_after_create = list_documents()
display(df_after_create)
counts.append(len(df_after_create))

print("\n✏️ Modification du document…")
# Trouver l'ID du dernier document ajouté
last_id = df_after_create['id_doc'].max()
print(f"   Modifying document ID={last_id}")
update_document(last_id, titre="Inflation 2024 (mise à jour)")
df_after_update = list_documents()
print("\n📊 Table 'document' après UPDATE :")
display(df_after_update)
# Afficher aussi le document complet modifié
df_full_update = get_document(last_id)
print("\n📄 Détails du document modifié :")
display(df_full_update)
counts.append(len(df_after_update))

print("\n🗑️ Suppression du document ID={}…".format(last_id))
delete_document(last_id)
df_after_delete = list_documents()
print("\n📊 Table 'document' finale après DELETE :")
display(df_after_delete)
counts.append(len(df_after_delete))

# 📋 Vue complète : Tous les documents restants avec tous les champs
print("\n📋 Vue complète de tous les documents restants :")
df_all = sql_df("SELECT * FROM document ORDER BY id_doc")
display(df_all)

# 📊 Visualisation : Évolution du nombre de documents pendant les opérations CRUD
plt.subplot(1, 2, 1)
plt.plot(stages, counts, marker='o', linewidth=2, markersize=10, color='#4ECDC4')
for i, (stage, count) in enumerate(zip(stages, counts)):
    plt.text(i, count + 0.05, str(count), ha='center', va='bottom', fontweight='bold')
plt.title("📊 Évolution du nombre de documents (CRUD)", fontsize=12, fontweight='bold')
plt.ylabel("Nombre de documents", fontsize=11)
plt.xticks(rotation=45, ha='right')
plt.grid(axis="y", linestyle="--", alpha=0.3)
plt.ylim(0, max(counts) + 1)

# Graphique 2 : Répartition par langue (état final)
plt.subplot(1, 2, 2)
if len(df_after_delete) > 0:
    lang_counts = df_after_delete['langue'].value_counts()
    colors = plt.cm.Set2(range(len(lang_counts)))
    bars = plt.bar(lang_counts.index, lang_counts.values, color=colors)
    for bar, value in zip(bars, lang_counts.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                str(value), ha='center', va='bottom', fontweight='bold')
    plt.title("📊 Répartition finale par langue", fontsize=12, fontweight='bold')
    plt.ylabel("Nombre de documents", fontsize=11)
    plt.grid(axis="y", linestyle="--", alpha=0.3)
else:
    plt.text(0.5, 0.5, "Aucun document", ha='center', va='center', fontsize=12)
    plt.title("📊 Répartition finale par langue", fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n✅ CRUD validé : {counts[0]} → {counts[1]} → {counts[2]} → {counts[3]} documents")
print("   • CREATE : +1 document")
print("   • UPDATE : Titre modifié")
print("   • DELETE : -1 document")
print("\n🎯 Toutes les opérations CRUD fonctionnent correctement !")


📄 Documents initiaux :


,id_doc,titre,langue,date_publication
0,1,JO 2024 : fierté nationale,fr,2024-07-27


➕ Ajout…


,id_doc,titre,langue,date_publication
0,1,JO 2024 : fierté nationale,fr,2024-07-27
1,2,Inflation 2024,fr,2024-09-15


✏️ Modification…


,id_doc,titre,langue,date_publication
0,1,JO 2024 : fierté nationale,fr,2024-07-27
1,2,Inflation 2024 (mise a jour),fr,2024-09-15


🗑️ Suppression…


,id_doc,titre,langue,date_publication
0,1,JO 2024 : fierté nationale,fr,2024-07-27


CRUD OK
